# Практическая работа №4: Элементы корреляционного анализа. Проверка статистической гипотезы о равенстве коэффициента корреляции нулю

Выполнили студенты гр. 0383 Желнин Максим, Рудакова Юлия, Сергевнин Дмитрий. Вариант №12


## Цель работы
Освоение основных понятий, связанных с корреляционной зависимостью между случайными величинами, статистическими гипотезами и проверкой их «справедливости».

## Основные теоретические положения



## Постановка задачи

Из заданной генеральной совокупности сформировать выборку по второму признаку. Провести статистическую обработку второй выборки в объёме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса и коэффициента вариации). Для системы двух случайных величин $X$ (первый признак) и $Y$ (второй признак) сформировать двумерную выборку и найти статистическую оценку коэффициента корреляции, построить доверительный интервал для коэффициента корреляции и осуществить проверку статистической гипотезы о равенстве коэффициента корреляции нулю. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы


### 1.
Провести статистическую обработку второй выборки в объеме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса, моды, медианы и коэффициента вариации). Оформить результаты в виде таблицы, сделать выводы.

In [1]:
import pandas as pd
import numpy as np
import io
import zipfile
from google.colab import files
import math
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt

In [2]:
uploaded = files.upload()

Saving sample.zip to sample.zip


In [3]:
with zipfile.ZipFile(io.BytesIO(uploaded['sample.zip']), 'r') as zip_ref:
    zip_ref.extractall()

df = pd.read_csv('sample.csv', skiprows=3) # двумерная генеральная совокупность
N = 111
sample = df.sample(n=N, random_state=2)
sample

,nu,E
94,406,113.8
32,542,146.1
225,382,98.1
157,521,154.9
356,405,107.5
...,...,...
128,429,120.9
176,470,124.0
108,412,127.8
222,383,109.7


In [4]:
ranked_nu = sample.sort_values(by='nu').reset_index(drop=True)['nu']
ranked_e = sample.sort_values(by='E').reset_index(drop=True)['E']

In [5]:
# Вычисляем среднеквадратичное отклонение
std_dev_nu = np.std(ranked_nu)
std_dev_e = np.std(ranked_e)

print("Среднеквадратичное отклонение для выборки ranked_nu:", std_dev_nu)
print("Среднеквадратичное отклонение для выборки ranked_e:", std_dev_e)

Среднеквадратичное отклонение для выборки ranked_nu: 59.02381081118964
Среднеквадратичное отклонение для выборки ranked_e: 23.87805236253307


In [6]:
# Преобразование ранжированного ряда в вариационный ряд для 'nu'
variation_nu = ranked_nu.value_counts().reset_index()
variation_nu.columns = ['значение', 'частота']
variation_nu_sort = variation_nu.sort_values(by='значение').reset_index(drop=True)
variation_nu_sort.drop(columns=['частота'], inplace=True)


# Преобразование ранжированного ряда в вариационный ряд для 'E'
variation_e = ranked_e.value_counts().reset_index()
variation_e.columns = ['значение', 'частота']
variation_e_sort = variation_e.sort_values(by='значение').reset_index(drop=True)
variation_e_sort.drop(columns=['частота'], inplace=True)

In [7]:
def create_interval_series(variation_series):
    # Определение интервалов
    num_bins = round(1 + 3.31 * math.log10(N))
    if num_bins % 2 == 0:
      num_bins += 1

    bin_width = (variation_series['значение'].max() - variation_series['значение'].min()) / num_bins
    bins = [variation_series['значение'].min() + i * bin_width for i in range(num_bins)]
    bins.append(variation_series['значение'].max())

    # Подсчет частоты попадания в интервалы
    frequency, _ = np.histogram(variation_series['значение'], bins=bins)

    # Определение границ и середин интервалов
    interval_bounds = [(bins[i], bins[i+1]) for i in range(num_bins)]
    interval_midpoints = [(interval[0] + interval[1]) / 2 for interval in interval_bounds]

    interval_series = pd.DataFrame({
        'Границы интервалов': interval_bounds,
        'Середины интервалов': interval_midpoints,
        'Частота попадания в интервал': frequency,
        'Относительная частота': frequency / len(variation_series)
    })

    return interval_series


interval_nu_series = create_interval_series(variation_nu_sort)
interval_e_series = create_interval_series(variation_e_sort)
print(interval_e_series['Относительная частота'])

0    0.057692
1    0.105769
2    0.096154
3    0.221154
4    0.192308
5    0.153846
6    0.076923
7    0.048077
8    0.048077
Name: Относительная частота, dtype: float64


In [8]:
def get_buckets(intervals, series):
    buckets = [[] for i in range(len(intervals) - 1)]
    for i in range(len(series) - 1):
      for j in range(len(intervals) - 1):
        if series[i] > intervals[j][0] and series[i] <= intervals[j][1]:
          buckets[j].append(series[i])
    return buckets

Для NU

In [9]:
mid_nu_values = interval_nu_series["Середины интервалов"]

buckets_nu = get_buckets(interval_nu_series["Границы интервалов"], variation_nu["значение"])
C_nu = mid_nu_values[max([(len(bucket), i) for i, bucket in enumerate(buckets_nu)])[1]]
h_nu = int((mid_nu_values[len(mid_nu_values) - 1] - mid_nu_values[0]) /(len(mid_nu_values)-1))
ui_arr_nu = [round((xi - C_nu) / h_nu) for xi in mid_nu_values]
nu_freq_arr = interval_nu_series["Частота попадания в интервал"]
len_intervals_nu = len(ui_arr_nu)

data_nu = {'xi': mid_nu_values,
        'ni': nu_freq_arr,
        'ui':  ui_arr_nu,
        'ui*n': [nu_freq_arr[i] * ui_arr_nu[i] for i in range(len_intervals_nu)],
        'ui^2 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 2) for i in range(len_intervals_nu)],
        'ui^3 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 3) for i in range(len_intervals_nu)],
        'ui^4 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 4) for i in range(len_intervals_nu)],
        '(ui + 1)^4 * n': [nu_freq_arr[i] * ((ui_arr_nu[i] + 1) ** 4) for i in range(len_intervals_nu)]}

table1_nu = pd.DataFrame(data_nu)
table1_nu.loc['Sum'] = table1_nu.sum()
print("Таблица 1:")
print(table1_nu)

Таблица 1:
              xi    ni   ui  ui*n  ui^2 * n  ui^3 * n  ui^4 * n  \
0     336.111111   4.0 -3.0 -12.0      36.0    -108.0     324.0   
1     366.333333   7.0 -2.0 -14.0      28.0     -56.0     112.0   
2     396.555556  14.0 -1.0 -14.0      14.0     -14.0      14.0   
3     426.777778  20.0  0.0   0.0       0.0       0.0       0.0   
4     457.000000  15.0  1.0  15.0      15.0      15.0      15.0   
5     487.222222  12.0  2.0  24.0      48.0      96.0     192.0   
6     517.444444   8.0  3.0  24.0      72.0     216.0     648.0   
7     547.666667   5.0  4.0  20.0      80.0     320.0    1280.0   
8     577.888889   6.0  5.0  30.0     150.0     750.0    3750.0   
Sum  4113.000000  91.0  9.0  73.0     443.0    1219.0    6335.0   

     (ui + 1)^4 * n  
0              64.0  
1               7.0  
2               0.0  
3              20.0  
4             240.0  
5             972.0  
6            2048.0  
7            3125.0  
8            7776.0  
Sum         14252.0  


Для Е

In [10]:
mid_e_values = interval_e_series["Середины интервалов"]

buckets_e = get_buckets(interval_e_series["Границы интервалов"], variation_e["значение"])
C_e = mid_e_values[max([(len(bucket), i) for i, bucket in enumerate(buckets_e)])[1]]
h_e = int((mid_e_values[len(mid_e_values) - 1] - mid_e_values[0]) /(len(mid_e_values)-1))
ui_arr_e = [(xi - C_e) / h_e for xi in mid_e_values]
e_freq_arr = interval_e_series["Частота попадания в интервал"]
len_intervals_e = len(ui_arr_e)

data_e = {'xi': mid_e_values,
        'ni': e_freq_arr,
        'ui':  ui_arr_e,
        'ui*n': [e_freq_arr[i] * ui_arr_e[i] for i in range(len_intervals_e)],
        'ui^2 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 2) for i in range(len_intervals_e)],
        'ui^3 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 3) for i in range(len_intervals_e)],
        'ui^4 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 4) for i in range(len_intervals_e)],
        '(ui + 1)^4 * n': [e_freq_arr[i] * ((ui_arr_e[i] + 1) ** 4) for i in range(len_intervals_e)]}

table1_e = pd.DataFrame(data_e)
table1_e.loc['Sum'] = table1_e.sum()
print("Таблица 1:")
print(table1_e)

Таблица 1:
              xi     ni        ui       ui*n    ui^2 * n     ui^3 * n  \
0      80.394444    6.0 -3.147222 -18.883333   59.430046  -187.039562   
1      92.983333   11.0 -2.098148 -23.079630   48.424482  -101.601738   
2     105.572222   10.0 -1.049074 -10.490741   11.005564   -11.545652   
3     118.161111   23.0  0.000000   0.000000    0.000000     0.000000   
4     130.750000   20.0  1.049074  20.981481   22.011128    23.091304   
5     143.338889   16.0  2.098148  33.570370   70.435610   147.784346   
6     155.927778    8.0  3.147222  25.177778   79.240062   249.386083   
7     168.516667    5.0  4.196296  20.981481   88.044513   369.460864   
8     181.105556    5.0  5.245370  26.226852  137.569552   721.603250   
Sum  1176.750000  104.0  9.441667  74.484259  516.160958  1211.138895   

        ui^4 * n  (ui + 1)^4 * n  
0     588.655067      127.543762  
1     213.175498       15.996922  
2      12.112244        0.000058  
3       0.000000       23.000000  
4      24.

In [11]:
nu_N = sum(data_nu["ni"])
vi_nu_arr = [sum(data_nu["ui*n"]) / nu_N, sum(data_nu["ui^2 * n"]) / nu_N, sum(data_nu["ui^3 * n"]) / nu_N, sum(data_nu["ui^4 * n"]) / nu_N]
mi_nu_arr = [0, (vi_nu_arr[1] - vi_nu_arr[0] ** 2) * (h_nu ** 2), (vi_nu_arr[2] - 3* vi_nu_arr[1] * vi_nu_arr[0]  + 2 * (vi_nu_arr[0]** 3)) * (h_nu ** 3), (vi_nu_arr[3] - 4 * vi_nu_arr[2] * vi_nu_arr[0] + 6 * vi_nu_arr[1] * (vi_nu_arr[0] ** 2)  - 3 * (vi_nu_arr[0]** 4)) * (h_nu ** 4)]
vi_e_arr = [sum(data_e["ui*n"]) / N, sum(data_e["ui^2 * n"]) / N, sum(data_e["ui^3 * n"]) / N, sum(data_e["ui^4 * n"]) / N]
mi_e_arr = [0, (vi_e_arr[1] - vi_e_arr[0] ** 2) * (h_e ** 2), (vi_e_arr[2] - 3* vi_e_arr[1] * vi_e_arr[0]  + 2 * (vi_e_arr[0]** 3)) * (h_e ** 3), (vi_e_arr[3] - 4 * vi_e_arr[2] * vi_e_arr[0] + 6 * vi_e_arr[1] * (vi_e_arr[0] ** 2)  - 3 * (vi_e_arr[0]** 4)) * (h_e ** 4)]


In [12]:
x_v2 = sum(ranked_e)/N

sigma2 = sum([(ranked_e[i] - x_v2) ** 2 for i in range(N)]) / N
print("Мат. ожидание: ", x_v2)
print("Дисперсия: ", sigma2)

Мат. ожидание:  126.72252252252252
Дисперсия:  570.1613846278709


Исправленная выборочная дисперсия $S^{2}$ и исправленное СКО S.

In [13]:
S_square = sigma2 * N / (N-1)
S = np.sqrt(S_square)
print(S_square, S)

575.3446699426697 23.986343405001726


Статистическая оценка коэффициентов асимметрии $a^{*}_{s}$ и эксцесса $ε^{*}_{s}$.

In [14]:
a_s = mi_e_arr[2] / (S ** 3)
e_s = mi_e_arr[3] / (S ** 4) - 3
print(a_s, e_s)

0.26976094290959585 0.016265496869106055


* Распределение имеет небольшую положительную асимметрию, что может указывать на наличие выбросов или аномалий в данных.
* Кривая распределения немного острая, но общее распределение не слишком отличается от нормального.

Самая высокая относительная частота у 3 интервала 0.221154


In [15]:
mode_value = data_e["xi"][3] + h_e * (interval_e_series['Относительная частота'][2]-interval_e_series['Относительная частота'][1])/(interval_e_series['Относительная частота'][2]-interval_e_series['Относительная частота'][1] + interval_e_series['Относительная частота'][2]-interval_e_series['Относительная частота'][3])
print(mode_value)

119.01825396825397


In [16]:
median_value = (ranked_e[N // 2] + ranked_e[N // 2 + 1]) / 2
print(median_value)

125.75


In [17]:
coefficient_variation_value = x_v2 / sigma2 * 100
print(coefficient_variation_value)

22.22572870402841


Коэффициент вариации = 22% => данные имеют относительно высокий уровень вариабельности или разброса относительно своего среднего значения

In [18]:
data = {
    'Параметр': ['Математическое ожидание', 'Дисперсия', 'Среднеквадратичное отклонение',
                 'Асимметрия', 'Эксцесс', 'Мода', 'Медиана', 'Коэффициент вариации'],
    'Значение': [x_v2, sigma2, std_dev_e, a_s, e_s,
                 mode_value, median_value, coefficient_variation_value]
}

df = pd.DataFrame(data)

print(df)

                        Параметр    Значение
0        Математическое ожидание  126.722523
1                      Дисперсия  570.161385
2  Среднеквадратичное отклонение   23.878052
3                     Асимметрия    0.269761
4                        Эксцесс    0.016265
5                           Мода  119.018254
6                        Медиана  125.750000
7           Коэффициент вариации   22.225729


Исходя из предоставленных статистических оценок параметров распределения исследуемого признака во второй выборке, можно сделать следующие выводы о его характере:

* Среднее значение (математическое ожидание) признака составляет примерно 126.72 единицы. Это указывает на то, что среднее значение находится вблизи центра распределения данных.

* Дисперсия составляет примерно 570.16. Это говорит о том, что значения признака относительно среднего значения имеют значительный разброс. Это может указывать на наличие различий между наблюдениями.

* Среднеквадратичное отклонение признака равно примерно 23.88. Это означает, что большинство значений признака находятся в пределах примерно плюс-минус 23.88 от среднего значения.

* Коэффициент асимметрии составляет примерно 0.27, что указывает на небольшую положительную асимметрию в данных. Это может означать, что распределение признака имеет длинный правый хвост.

* Эксцесс близок к нулю (0.02), что указывает на то, что пик распределения немного острый, но в целом распределение признака близко к нормальному.

* Мода признака составляет примерно 119.02. Это значение является наиболее часто встречающимся в выборке.

Исходя из этих данных, можно сделать вывод о том, что распределение исследуемого признака во второй выборке имеет некоторую положительную асимметрию с длинным правым хвостом, но в целом близко к нормальному распределению. Среднее значение и мода находятся вблизи друг от друга, а значения признака имеют некоторый разброс относительно среднего.

2.
Построить двумерный интервальный вариационный ряд, оформить в виде таблицы.

In [19]:
interval_nu = [str(interval) for interval in interval_nu_series['Границы интервалов']]
for i,e in enumerate(interval_nu):
  interval_nu[i] = e.replace("(", "[")
  if i == len(interval_nu) - 1:
    interval_nu[i] = e.replace(")", "]").replace("(", "[")

interval_e = [str(interval) for interval in interval_e_series['Границы интервалов']]
for i,e in enumerate(interval_e):
  interval_e[i] = e.replace("(", "[")
  if i == len(interval_e) - 1:
    interval_e[i] = e.replace(")", "]").replace("(", "[")
print(interval_nu)
print(interval_e)

['[321.0, 351.22222222222223)', '[351.22222222222223, 381.44444444444446)', '[381.44444444444446, 411.66666666666663)', '[411.66666666666663, 441.8888888888889)', '[441.8888888888889, 472.1111111111111)', '[472.1111111111111, 502.3333333333333)', '[502.3333333333333, 532.5555555555555)', '[532.5555555555555, 562.7777777777778)', '[562.7777777777778, 593]']
['[74.1, 86.68888888888888)', '[86.68888888888888, 99.27777777777777)', '[99.27777777777777, 111.86666666666667)', '[111.86666666666667, 124.45555555555555)', '[124.45555555555555, 137.04444444444445)', '[137.04444444444445, 149.63333333333333)', '[149.63333333333333, 162.22222222222223)', '[162.22222222222223, 174.81111111111113)', '[174.81111111111113, 187.4]']


In [20]:
two_dimensional_variational_series = pd.DataFrame(index=interval_nu, columns=interval_e)
two_dimensional_variational_series.fillna(0, inplace=True)

for _, row in sample.iterrows():
    nu_interval = None
    E_interval = None

    # Находим интервалы, в которые попадают данные из sample для nu и E
    for index, interval in enumerate(interval_nu_series['Границы интервалов']):
        if interval[0] <= row['nu'] < interval[1]:
            nu_interval = interval_nu[index]
            break
        elif index == len(interval_nu_series['Границы интервалов']) - 1:
            nu_interval = interval_nu[index]

    for index, interval in enumerate(interval_e_series['Границы интервалов']):
        if interval[0] <= row['E'] < interval[1]:
            E_interval = interval_e[index]
            break
        elif index == len(interval_e_series['Границы интервалов']) - 1:
            nu_interval = interval_e[index]
    if nu_interval is not None and E_interval is not None:
        two_dimensional_variational_series[E_interval][nu_interval] += 1

print("Двумерный интервальный вариационный ряд:")
two_dimensional_variational_series


Двумерный интервальный вариационный ряд:


,"[74.1, 86.68888888888888)","[86.68888888888888, 99.27777777777777)","[99.27777777777777, 111.86666666666667)","[111.86666666666667, 124.45555555555555)","[124.45555555555555, 137.04444444444445)","[137.04444444444445, 149.63333333333333)","[149.63333333333333, 162.22222222222223)","[162.22222222222223, 174.81111111111113)","[174.81111111111113, 187.4]"
"[321.0, 351.22222222222223)",2,2,1,0,0,0,0,0,0
"[351.22222222222223, 381.44444444444446)",2,4,1,0,0,0,0,0,0
"[381.44444444444446, 411.66666666666663)",2,4,5,3,0,0,0,0,0
"[411.66666666666663, 441.8888888888889)",0,1,3,16,11,0,0,0,0
"[441.8888888888889, 472.1111111111111)",0,0,0,5,9,5,0,0,0
"[472.1111111111111, 502.3333333333333)",0,0,0,1,4,8,0,0,0
"[502.3333333333333, 532.5555555555555)",0,0,0,0,0,3,6,2,0
"[532.5555555555555, 562.7777777777778)",0,0,0,0,0,1,1,2,1
"[562.7777777777778, 593]",0,0,0,0,0,0,1,1,3


3.
По полученному двумерному интервальному вариационному ряду построить корреляционную таблицу.

In [21]:
two_dimensional_variational_series['Сумма'] = two_dimensional_variational_series.sum(axis=1)

In [22]:
new_row = [two_dimensional_variational_series[col].sum() for col in two_dimensional_variational_series.columns]
# new_row.insert(0, 'Сумма')
new_row


[6, 11, 10, 25, 24, 17, 8, 5, 4, 110]

In [23]:
two_dimensional_variational_series.loc['Сумма'] = new_row

two_dimensional_variational_series

,"[74.1, 86.68888888888888)","[86.68888888888888, 99.27777777777777)","[99.27777777777777, 111.86666666666667)","[111.86666666666667, 124.45555555555555)","[124.45555555555555, 137.04444444444445)","[137.04444444444445, 149.63333333333333)","[149.63333333333333, 162.22222222222223)","[162.22222222222223, 174.81111111111113)","[174.81111111111113, 187.4]",Сумма
"[321.0, 351.22222222222223)",2,2,1,0,0,0,0,0,0,5
"[351.22222222222223, 381.44444444444446)",2,4,1,0,0,0,0,0,0,7
"[381.44444444444446, 411.66666666666663)",2,4,5,3,0,0,0,0,0,14
"[411.66666666666663, 441.8888888888889)",0,1,3,16,11,0,0,0,0,31
"[441.8888888888889, 472.1111111111111)",0,0,0,5,9,5,0,0,0,19
"[472.1111111111111, 502.3333333333333)",0,0,0,1,4,8,0,0,0,13
"[502.3333333333333, 532.5555555555555)",0,0,0,0,0,3,6,2,0,11
"[532.5555555555555, 562.7777777777778)",0,0,0,0,0,1,1,2,1,5
"[562.7777777777778, 593]",0,0,0,0,0,0,1,1,3,5
Сумма,6,11,10,25,24,17,8,5,4,110


In [ ]:
import numpy as np
from scipy.special import erf

In [ ]:
laplas = lambda x: erf(x/2**0.5)/2

In [ ]:
N = 111
X_avg = 128.536
S = 24.3566
y_1 = 0.95
y_2 = 0.99
t_accuracy_1 = 1.9855;
t_accuracy_2 = 2.627;

t = 1.9855 при γ = 0.95 и n = N-1 = 110

In [ ]:
accuracy_1 = t_accuracy_1 * S / np.sqrt(N)
a_left_1 = X_avg - accuracy_1
a_right_1 = X_avg + accuracy_1
print("Доверительный интервал для математического ожидания при y=0.95 (", a_left_1, " ; ", a_right_1, ")")
print("Точность ", accuracy_1)
accuracy_2 = t_accuracy_2 * S / np.sqrt(N)
a_left_2 = X_avg - accuracy_2
a_right_2 = X_avg + accuracy_2
print("Доверительный интервал для математического ожидания при y=0.99 (", a_left_2, " ; ", a_right_2, ")")
print("Точность ", accuracy_2)

Доверительный интервал для математического ожидания при y=0.95 ( 123.94586915150799  ;  133.126130848492 )
Точность  4.590130848492013
Доверительный интервал для математического ожидания при y=0.99 ( 122.46283266734399  ;  134.60916733265603 )
Точность  6.0731673326560145


Вывод:

Мат. ожидание с вероятностью 0.95 в интервале ( 123.94586915150799  ;  133.126130848492 )

Мат. ожидание с вероятностью 0.99 в интервале ( 122.46283266734399  ;  134.60916733265603 )

Второй интервал более вероятен, но и имеет больший диапозон, следовательно не всегда информативен

2. Для вычисления границ доверительного интервала для среднеквадратичного отклонения определить значение q при заданных γ и  n. Построить доверительные интервалы, сделать выводы.

In [ ]:
q = 0.143
sd_left = S*(1-q)
sd_right = S*(1+q)
print("q = ", q)
print("Доверительный интервал для среднеквадратического отклонения: ")
print(sd_left, sd_right)

q =  0.143
Доверительный интервал для среднеквадратического отклонения: 
20.8736062 27.8395938


Из таблицы q = 0.143


Вывод:

С вероятностью 0.95 СКВО лежит в интервале (20.8736062; 27.8395938)

3.
Проверить гипотезу о нормальности заданного распределения с помощью критерия χ2 (Пирсона). Для этого необходимо найти теоретические частоты и вычислить наблюдаемое значение критерия. Для удобства вычисления необходимо заполнить табл. 1.

H0 - гипотеза о том, что выборка имеет нормальное распределение, H1 - альтернативная гипотеза, что выборка имеет не нормальное распределение.

In [ ]:
freq = [13, 13, 13, 19, 18, 17, 7, 6]
mid_values = [84, 102, 114, 126, 138, 150, 162, 174]
borders = [[72.6, 96.6, 108.7, 120.7, 132.8, 144.8, 156.9, 168.9],[96.6, 108.7, 120.7, 132.8, 144.8, 156.9, 168.9, 181.0]]
z = [[(e - X_avg)/S for e in borders[0]],[(i - X_avg)/S for i in borders[1]]]
z[0][0] = -float('inf')
z[1][-1] = float('inf')
p = []
for i in range(0, len(z[0])):
    p.append(laplas(z[1][i])-laplas(z[0][i]))
freq_theor = [x*sum(freq) for x in p]

In [ ]:
print("(x_i-1, x_i):")
for i in range(len(borders[0])):
  print(borders[0][i], borders[1][i])
print("n_i")
print(freq)
print("p_i")
print(p)
print("n_i'")
print(freq_theor)
print("(n_i-n_i')^2")
print([(freq_theor[i] - freq[i])**2 for i in range(len(freq))])
print("(n_i-n_i')^2/n_i'")
print([(freq_theor[i] - freq[i])**2 / freq_theor[i] for i in range(len(freq))])
print("n_i^2")
print([freq[i]**2 for i in range(len(freq))])
print("n_i^2/n_i'")
print([freq[i]**2 / freq_theor[i] for i in range(len(freq))])

(x_i-1, x_i):
72.6 96.6
96.6 108.7
108.7 120.7
120.7 132.8
132.8 144.8
144.8 156.9
156.9 168.9
168.9 181.0
n_i
[13, 13, 13, 19, 18, 17, 7, 6]
p_i
[0.09489769385540875, 0.11281040228097877, 0.16612439957590025, 0.19565341522520005, 0.17836592704796528, 0.13004358703510846, 0.07336605773131383, 0.04873851724812461]
n_i'
[10.059155548673328, 11.95790264178375, 17.609186355045427, 20.739262013871205, 18.90678826708432, 13.784620225721497, 7.776802119519266, 5.166282828301209]
(n_i-n_i')^2
[8.648566086898876, 1.0859669040012865, 21.24459885553695, 3.0250323528953187, 0.8222649613217835, 10.338667092839277, 0.6034215328896234, 0.6950843223854317]
(n_i-n_i')^2/n_i'
[0.8597705885997071, 0.09081583422553219, 1.206449771567662, 0.14586017336933504, 0.043490462245949074, 0.7500146484665408, 0.07759250185562454, 0.1345424448266204]
n_i^2
[169, 169, 169, 361, 324, 289, 49, 36]
n_i^2/n_i'
[16.80061503992638, 14.132913192441782, 9.597263416522235, 17.40659815949813, 17.13670219516163, 20.965394422745

k = K - 3 = 5

$χ2_{крит} = 11.1$ из таблицы

In [ ]:
k = 5
chi2_expected = 11.1

In [ ]:
stats_crit = []
for i in range(len(freq)):
    stats_crit.append((freq[i] - freq_theor[i])**2/freq_theor[i])
chi2_calculated = sum(stats_crit)
print("Наблюдаемое значение критерия: ", chi2_calculated)
print("Наблюдаемое < критическое: ", chi2_calculated < chi2_expected)

Наблюдаемое значение критерия:  3.3085364251569715
Наблюдаемое < критическое:  True


Основная гипотеза подтверждена

4.

$χ^2_{набл} = \sum\frac{(n_i - n^`_i)^2}{n_i^`} = \sum\frac{n_i^2 - 2n_in^`_i + (n^`_i)^2}{n_i^`} = \sum\frac{n_i^2}{n_i^`} - 2\sum n_i + \sum n^`_i = \sum\frac{n_i^2}{n_i^`} - 2N + N = \sum\frac{n_i^2}{n_i^`} - N$

In [ ]:
print(sum([(freq[i]**2) / freq_theor[i] for i in range(len(freq))]) - sum(freq))

3.308536425156987


Сошлось с посчитанным ранее

5. Сделано в п.3, так как наблюдаемое значение критерия ниже критического, то гипотеза о нормальности распределения подтверждена.

##Вывод


В процессе выполнения практической работы были рассчитаны точность и доверительный интервал для математического ожидания при неизвестном стандартном отклонении для уровней доверия 0.95, 0.99, а также было определено значение q. Потом при помощи критерия $χ^2$ была проверена гипотеза о типа распределения выборки и было получено, что выборка имеет нормальное распределение. Для этой проверки были вычислены теоретические частоты и определено наблюдаемое значение критерия. Было также доказано равенство двух формул для расчета $χ^2_{набл}$.